In [1]:
import copy
import xarray as xr
import numpy as np
import pathlib
import time

# Filepaths

In [2]:
## Path to server
server_fp = pathlib.Path("/Volumes")

## Filepath to the CESM2 historical model output
cesm_fp = pathlib.Path("cmip6/data/cmip6/CMIP/NCAR/CESM2/historical/r1i1p1f1")

## filepath to data
cesm_fp_pre = server_fp / cesm_fp / pathlib.Path("Amon/pr/gn/1")

# Load data and trim to Pacific

In [3]:
## open data
cesm_hist_pre = xr.open_mfdataset(cesm_fp_pre.glob("*.nc"))

## trim to area
cesm_hist_pre = cesm_hist_pre["pr"].sel(lat=slice(-30, 30), lon=slice(100, 300))

/Users/theo/research/12860_tutorial/envs/lib/python3.11/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'pr' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


# Do the conversion (and time it)

In [4]:
## get number of days per month
num_days = cesm_hist_pre.time.dt.days_in_month

## Create new array with updated units (don't do the computation yet, though)
cesm_hist_pre_m = cesm_hist_pre * num_days * 24 * 60 * 60 / 1000

## Do the computation and time it
t0 = time.time()
cesm_hist_pre_m.load()
t1 = time.time()

print(f"Elapsed time: {t1-t0:.2f} seconds")

Elapsed time: 84.41 seconds


# Check it gives same result as loop

In [19]:
lat_idx, lon_idx = 30, 101
time = "2001-02"
conversion_factor = 28 * (24 * 60 * 60 / 1000)

print(
    np.allclose(
        cesm_hist_pre.isel(lat=lat_idx, lon=lon_idx).sel(time=time).compute()
        * conversion_factor,
        cesm_hist_pre_m.isel(lat=lat_idx, lon=lon_idx).sel(time=time),
    )
)

True
